In [1]:
import calibrators
import simulators
import logging
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
logging.basicConfig(filename='data/app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s', level=logging.DEBUG)

In [3]:
def print_model_status(calibrator, model):
    print('model parameters: ' + str(model.params))
    print('calibrator model parameters: ' + str(calibrator.model_params))
    print('model parameters difference: ' + str((np.asarray(model.params) - np.asarray(calibrator.model_params)).tolist()))
    print('\nsamping point: ' + str(calibrator.sample_queue[-1].inputs))
    print('sampling value: ' + str(calibrator.sample_queue[-1].output))
    print('current observe: ' + str(model.observe(calibrator.sample_queue[-1].inputs)))

# Initialize Model and Calibrator

### Initial Model Parameters

In [4]:
model_params = [0.7991353, 2.2477762]

### Initialize

In [5]:
model = simulators.SimulatorModel(list(model_params))
calib = calibrators.PointLSCalibrator(model, num_resamples=1, num_samples=12)
calib.initialize()

initializing...
initialization done!
`xtol` termination condition is satisfied.
Function evaluations 18, initial cost 5.6624e-01, final cost 3.1834e-18, first-order optimality 3.81e-09.


/Users/keyit/miniconda3/envs/polarization/lib/python2.7/site-packages/scipy/optimize/_lsq/least_squares.py:114: UserWarning: `xtol` is too low, setting to machine epsilon 2.22044604925e-16.
  warn(message.format("`xtol`", EPS))
simulators.py:140: OptimizeWarning: Unknown solver options: gtol
  options={'gtol': 1e-6, 'disp': verbose > 0, 'xtol': 1e-15}


In [6]:
print_model_status(calib, model)

model parameters: [0.7991353, 2.2477762]
calibrator model parameters: [ 0.7991353  2.2477762]
model parameters difference: [6.854667944367066e-10, -2.258059073056984e-09]

samping point: [2.4646128029146057, 0.92945520328508757]
sampling value: 1.38777878078e-16
current observe: 1.38777878078e-16


# Tracking

In [7]:
model.params = list(model_params)
calib.model_params = list(model_params)

### Add a Small Perturbation

In [8]:
params_change = [1E-3, 5E-2]

In [9]:
for i in range(len(model.params)):
    model.params[i] = model_params[i] + params_change[i]

In [10]:
print_model_status(calib, model)

model parameters: [0.8001353, 2.2977762]
calibrator model parameters: [0.7991353, 2.2477762]
model parameters difference: [0.0010000000000000009, 0.04999999999999982]

samping point: [2.4646128029146057, 0.92945520328508757]
sampling value: 1.38777878078e-16
current observe: 0.000402022655257


### Track

In [11]:
calib.track(num_observes=5)

/Users/keyit/miniconda3/envs/polarization/lib/python2.7/site-packages/numpy/__init__.py:1: RuntimeWarning: invalid value encountered in sqrt
  """
/Users/keyit/miniconda3/envs/polarization/lib/python2.7/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [12]:
print_model_status(calib, model)

model parameters: [0.8001353, 2.2977762]
calibrator model parameters: [0.7991353, 2.2477762]
model parameters difference: [0.0010000000000000009, 0.04999999999999982]

samping point: [2.4646128029146057, 0.92945520328508757]
sampling value: 1.38777878078e-16
current observe: 0.000402022655257


# Simulation